# Programmatic Prompting

In [3]:

from litellm import completion
from dotenv import load_dotenv

load_dotenv()

def generate_response(message_list: list[dict]) -> str:
    """Call LLM to get response"""
    r = completion(
        model="openai/gpt-4o",
        messages=message_list,
        max_tokens=1024
    )
    return r.choices[0].message.content


messages = [
    {"role": "system", "content": "You are an expert software engineer that prefers functional programming."},
    {"role": "user", "content": "Write a function to swap the keys and values in a dictionary."}
]

response = generate_response(messages)
print(response)

To swap the keys and values in a dictionary using a functional programming approach, you can utilize Python's built-in functions to create a new dictionary from the original one. Here's a concise way to achieve this using a dictionary comprehension:

```python
def swap_dict_keys_values(input_dict):
    # Using dictionary comprehension to swap keys and values
    return {value: key for key, value in input_dict.items()}

# Example usage:
original_dict = {'a': 1, 'b': 2, 'c': 3}
swapped_dict = swap_dict_keys_values(original_dict)
print(swapped_dict)  # Output: {1: 'a', 2: 'b', 3: 'c'}
```

### Important Considerations:

1. **Value Uniqueness**: Ensure that all values in the original dictionary are unique. If there are duplicate values, swapping will result in the loss of some data since dictionaries cannot have duplicate keys.

2. **Immutability**: In a functional programming style, the function does not modify the input dictionary but returns a new dictionary with the keys and values swa

In [4]:
import json

code_spec = {
    'name': 'swap_keys_values',
    'description': 'Swaps the keys and values in a given dictionary.',
    'params': {
        'd': 'A dictionary with unique values.'
    },
}

messages = [
    {"role": "system",
     "content": "You are an expert software engineer that writes clean functional code. You always document your functions."},
    {"role": "user", "content": f"Please implement: {json.dumps(code_spec)}"}
]

response = generate_response(messages)
print(response)

```python
def swap_keys_values(d):
    """
    Swaps the keys and values in a given dictionary.

    Parameters:
    d (dict): A dictionary with unique values.

    Returns:
    dict: A new dictionary with keys and values swapped.
    
    Raises:
    ValueError: If the input dictionary has non-unique values.
    
    Example:
    >>> swap_keys_values({'a': 1, 'b': 2, 'c': 3})
    {1: 'a', 2: 'b', 3: 'c'}
    """
    
    # Check if the values are unique
    if len(d.values()) != len(set(d.values())):
        raise ValueError("The dictionary contains non-unique values")

    # Swap keys and values
    return {value: key for key, value in d.items()}

# Example Usage
# swapped_dict = swap_keys_values({'a': 1, 'b': 2, 'c': 3})
# print(swapped_dict)
```

This function `swap_keys_values` takes a dictionary as input, checks if the values are unique, and then returns a new dictionary with keys and values swapped.



# Customer service agent

In [5]:
what_to_help_with = input("What do you need help with?")

messages = [
    {"role": "system", "content": "You are a helpful customer service representative. No matter what the user asks, the solution is to tell them to turn their computer or modem off and then back on."},
    {"role": "user", "content": what_to_help_with}
]

response = generate_response(messages)
print(response)

It sounds like you might be having some technical issues. I recommend turning your computer or modem off and then back on. If you're looking for help finding apples, a fresh start might clear up any browser issues or internet search difficulties!


# Quasi-Agent

In [12]:
import sys

def extract_code_block(response: str) -> str:
   """Extract code block from response"""

   if not '```' in response:
      return response

   code_block = response.split('```')[1].strip()
   # Check for "python" at the start and remove

   if code_block.startswith("python"):
      code_block = code_block[6:]

   return code_block

def develop_custom_function():
   # Get user input for function description
   print("\nWhat kind of function would you like to create?")
   print("Example: 'A function that calculates the factorial of a number'")
   print("Your description: ", end='')
   function_description = input().strip()

   # Initialize conversation with system prompt
   messages = [
      {
          "role": "system",
          "content": "You are a Python expert helping to develop a function."
      },
      {
          "role": "user",
          "content": f"Write a Python function that {function_description}. Output the function in a ```python code block```."
      }
   ]

   # First prompt - Basic function
   initial_function = generate_response(messages)

   # Parse the response to get the function code
   initial_function = extract_code_block(initial_function)

   print("\n=== Initial Function ===")
   print(initial_function)

   # Add assistant's response to conversation
   # Notice that I am purposely causing it to forget its commentary and just see the code so that
   # it appears that is always outputting just code.
   messages.append(
       {
           "role": "assistant",
           "content": f"```python\n\n{initial_function}\n\n```"
       }
   )

   # Second prompt - Add documentation
   messages.append(
       {
           "role": "user",
           "content": "Add comprehensive documentation to this function, including description, parameters, "
                      "return value, examples, and edge cases. Output the function in a ```python code block```."
       }
   )
   documented_function = generate_response(messages)
   documented_function = extract_code_block(documented_function)
   print("\n=== Documented Function ===")
   print(documented_function)

   # Add documentation response to conversation
   messages.append(
       {
           "role": "assistant",
           "content": f"```python\n\n{documented_function}\n\n```"
       }
   )

   # Third prompt - Add test cases
   messages.append(
       {
           "role": "user",
           "content": "Add unittest test cases for this function, including tests for basic functionality, "
                      "edge cases, error cases, and various input scenarios. Output the code in a ```python code block```."
       }
   )
   test_cases = generate_response(messages)
   # We will likely run into random problems here depending on if it outputs JUST the test cases or the
   # test cases AND the code. This is the type of issue we will learn to work through with agents in the course.
   test_cases = extract_code_block(test_cases)
   print("\n=== Test Cases ===")
   print(test_cases)

   # Generate filename from function description
   filename = function_description.lower()
   filename = ''.join(c for c in filename if c.isalnum() or c.isspace())
   filename = filename.replace(' ', '_')[:30] + '.py'

   # Save final version
   with open(filename, 'w') as f:
      f.write(documented_function + '\n\n' + test_cases)

   return documented_function, test_cases, filename

if __name__ == "__main__":
   function_code, tests, filename = develop_custom_function()
   print(f"\nFinal code has been saved to {filename}")


What kind of function would you like to create?
Example: 'A function that calculates the factorial of a number'
Your description: 
=== Initial Function ===

def fibonacci_up_to_n(n):
    """Calculate Fibonacci numbers up to a given number n.

    Args:
        n (int): The upper limit for the Fibonacci sequence.

    Returns:
        list: A list containing all Fibonacci numbers up to n.
    """
    if n < 0:
        return []

    fib_seq = []
    a, b = 0, 1

    while a <= n:
        fib_seq.append(a)
        a, b = b, a + b

    return fib_seq

# Example usage
print(fibonacci_up_to_n(21))

=== Documented Function ===

def fibonacci_up_to_n(n):
    """
    Calculate Fibonacci numbers up to a given number n.

    This function generates all Fibonacci numbers that are less than or equal to the specified 
    upper limit `n`. The Fibonacci sequence starts with 0 and 1, and each subsequent number 
    is the sum of the two preceding numbers. The sequence produced will include numbers l

# Building a simple agent

In [10]:
import os
from litellm import completion
from dotenv import load_dotenv

load_dotenv()

def generate_response(message_list: list[dict]) -> str:
    """Call LLM to get response"""
    r = completion(
        model="openai/gpt-4o",
        messages=message_list,
        max_tokens=1024
    )
    return r.choices[0].message.content


def extract_markdown_block(response: str, block_type: str = "json") -> str:
    """Extract code block from response"""

    if not '```' in response:
        return response

    code_block = response.split('```')[1].strip()

    if code_block.startswith(block_type):
        code_block = code_block[len(block_type):].strip()

    return code_block


def parse_action(response: str) -> dict:
    """Parse the LLM response into a structured action dictionary."""
    try:
        response = extract_markdown_block(response, "action")
        response_json = json.loads(response)
        if "tool_name" in response_json and "args" in response_json:
            return response_json
        else:
            return {"tool_name": "error", "args": {"message": "You must respond with a JSON tool invocation."}}
    except json.JSONDecodeError:
        return {"tool_name": "error", "args": {"message": "Invalid JSON response. You must respond with a JSON tool invocation."}}


def list_files() -> list[str]:
    """List files in the current directory."""
    return os.listdir(".")


def read_file(file_name: str) -> str:
    """Read a file's contents."""
    try:
        with open(file_name, "r") as file:
            return file.read()
    except FileNotFoundError:
        return f"Error: {file_name} not found."
    except Exception as e:
        return f"Error: {str(e)}"


In [12]:
# Define system instructions (Agent Rules)
agent_rules = [{
    "role": "system",
    "content": """
You are an AI agent that can perform tasks by using available tools.

Available tools:

```json
{
    "list_files": {
        "description": "Lists all files in the current directory.",
        "parameters": {}
    },
    "read_file": {
        "description": "Reads the content of a file.",
        "parameters": {
            "file_name": {
                "type": "string",
                "description": "The name of the file to read."
            }
        }
    },
    "terminate": {
        "description": "Ends the agent loop and provides a summary of the task.",
        "parameters": {
            "message": {
                "type": "string",
                "description": "Summary message to return to the user."
            }
        }
    }
}
```

If a user asks about files, documents, or content, first list the files before reading them.

When you are done, terminate the conversation by using the "terminate" tool and I will provide the results to the user.

Important!!! Every response MUST have an action.
You must ALWAYS respond in this format:

<Stop and think step by step. Parameters map to args. Insert a rich description of your step by step thoughts here.>

```action
{
    "tool_name": "insert tool_name",
    "args": {...fill in any required arguments here...}
}
```"""
}]

In [13]:
import json


MAX_ITERATIONS: int = 10

user_task = input("What would you like me to do? ")

memory: list[dict[str, str]] = [{"role": "user", "content": user_task}]

# The Agent Loop
for iteration in range(MAX_ITERATIONS):

    # 1. Construct prompt: Combine agent rules with memory
    prompt = agent_rules + memory

    # 2. Generate response from LLM
    print("Agent thinking...")
    response = generate_response(prompt)
    print(f"Agent response: {response}")

    # 3. Parse response to determine action
    action = parse_action(response)

    result = "Action executed"

    if action["tool_name"] == "list_files":
        result = {"result":list_files()}
    elif action["tool_name"] == "read_file":
        result = {"result":read_file(action["args"]["file_name"])}
    elif action["tool_name"] == "error":
        result = {"error":action["args"]["message"]}
    elif action["tool_name"] == "terminate":
        print(action["args"]["message"])
        break
    else:
        result = {"error":"Unknown action: "+action["tool_name"]}

    print(f"Action result: {result}")

    # 5. Update memory with response and results
    memory.extend([
        {"role": "assistant", "content": response},
        {"role": "user", "content": json.dumps(result)}
    ])

    # 6. Check termination condition
    if action["tool_name"] == "terminate":
        break


Agent thinking...
Agent response: To list the files in the current directory, I will use the `list_files` tool to fetch and display the available files.

```action
{
    "tool_name": "list_files",
    "args": {}
}
```
Action result: {'result': ['calculate_fibonacci_numbers_up.py', 'ProgrammaticPrompting1.ipynb']}
Agent thinking...
Agent response: There are two files in the current directory: `calculate_fibonacci_numbers_up.py` and `ProgrammaticPrompting1.ipynb`. If you want to know more about the content of any of these files, please let me know which one you are interested in, and I will proceed to read it.

```action
{
    "tool_name": "terminate",
    "args": {
        "message": "The files in the directory are 'calculate_fibonacci_numbers_up.py' and 'ProgrammaticPrompting1.ipynb'. If you need details about a specific file, let me know."
    }
}
```
The files in the directory are 'calculate_fibonacci_numbers_up.py' and 'ProgrammaticPrompting1.ipynb'. If you need details about a spec

# Using LLM function calling for structured execution

In [2]:
import json
import os
from litellm import completion


def list_files(directory: str = ".") -> list[str]:
    """List files in the current directory."""
    return os.listdir(directory)


def read_file(file_name: str) -> str:
    """Read a file's contents."""
    try:
        with open(file_name, "r") as file:
            return file.read()
    except FileNotFoundError:
        return f"Error: {file_name} not found."
    except Exception as e:
        return f"Error: {str(e)}"


def terminate(message: str) -> None:
    """Terminate the agent loop and provide a summary message."""
    print(f"Termination message: {message}")


tool_functions = {
    "list_files": list_files,
    "read_file": read_file,
    "terminate": terminate
}

tools = [
    {
        "type": "function",
        "function": {
            "name": "list_files",
            "description": "Returns a list of files in the directory.",
            "parameters": {
                "type": "object",
                "properties": {"directory": {"type": "string", "default": "."}},
                "required": []
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "read_file",
            "description": "Reads the content of a specified file in the directory.",
            "parameters": {
                "type": "object",
                "properties": {"file_name": {"type": "string"}},
                "required": ["file_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "terminate",
            "description": "Terminates the conversation. No further actions or interactions are possible after this. Prints the provided message for the user.",
            "parameters": {
                "type": "object",
                "properties": {
                    "message": {"type": "string"},
                },
                "required": ["message"]
            }
        }
    }
]

# Our rules are simplified since we don't have to worry about getting a specific output format
agent_rules = [{
    "role": "system",
    "content": """
You are an AI agent that can perform tasks by using available tools.

If a user asks about files, documents, or content, first list the files before reading them.

When you are done, terminate the conversation by using the "terminate" tool and I will provide the results to the user.
"""
}]

In [4]:
MAX_ITERATIONS: int = 10

user_task = input("What would you like me to do? ")

memory: list[dict[str, str]] = [{"role": "user", "content": user_task}]

# The Agent Loop
for iteration in range(MAX_ITERATIONS):
    messages = agent_rules + memory

    response = completion(
        model="openai/gpt-4o",
        messages=messages,
        tools=tools,
        max_tokens=1024
    )

    if response.choices[0].message.tool_calls:
        tool = response.choices[0].message.tool_calls[0]
        tool_name = tool.function.name
        tool_args = json.loads(tool.function.arguments)

        action = {
            "tool_name": tool_name,
            "args": tool_args
        }

        if tool_name == "terminate":
            print(f"Termination message: {tool_args['message']}")
            break
        elif tool_name in tool_functions:
            try:
                result = {"result": tool_functions[tool_name](**tool_args)}
            except Exception as e:
                result = {"error":f"Error executing {tool_name}: {str(e)}"}
        else:
            result = {"error": f"Unknown tool: {tool_name}"}

        print(f"Executing: {tool_name} with args {tool_args}")
        print(f"Result: {result}")
        memory.extend([
            {"role": "assistant", "content": json.dumps(action)},
            {"role": "user", "content": json.dumps(result)}
        ])
    else:
        result = response.choices[0].message.content
        print(f"Response: {result}")
        break

Executing: list_files with args {'directory': 'test_dir'}
Result: {'result': ['file1']}
Termination message: The directory 'test_dir' contains one file: 'file1'.
